In [121]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import seaborn as sn
import pandas as pd
import altair as alt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import matplotlib.pyplot as plt
import matplotlib
import pandas_profiling
import scipy
import pylab
import io
import math
# Comment this if the data visualisations doesn't work on your side
%matplotlib inline
plt.style.use('bmh')

# Client List

#### Get clients from  market and add  label:
* 1 for Clients from Portfolio 1
* 2 for Clients from Portfolio 2
* 3 for Clients from Portfolio 3
* 0 for Clients from none above. 

In [122]:
market = pd.read_csv("estaticos_market.csv")
market['portfolio'] = 0
market.head()

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,...,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0,0
1,1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [123]:
data1 = pd.read_csv("estaticos_portfolio1.csv")
market.loc[market['id'].isin(data1['id']),'portfolio'] = 1

In [124]:
data2 = pd.read_csv("estaticos_portfolio2.csv")
market.loc[market['id'].isin(data2['id']),'portfolio'] = 2

In [125]:
data3 = pd.read_csv("estaticos_portfolio3.csv")
market.loc[market['id'].isin(data3['id']),'portfolio'] = 3

In [126]:
print("Market: Qtde de clientes: %d features: %d "%(market[market['portfolio']==0].shape))
print("Portfolio1: Qtde de clientes: %d features: %d "%(market[market['portfolio']==1].shape))
print("Portfolio2: Qtde de clientes: %d features: %d "%(market[market['portfolio']==2].shape))
print("Portfolio3: Qtde de clientes: %d features: %d "%(market[market['portfolio']==3].shape))

Market: Qtde de clientes: 461020 features: 183 
Portfolio1: Qtde de clientes: 555 features: 183 
Portfolio2: Qtde de clientes: 458 features: 183 
Portfolio3: Qtde de clientes: 265 features: 183 


# General Information

### Total of missing values by features

* Drop features with missing values greater than 85% of total.

In [306]:
row, col = market.shape
print("%d rows" %(row))
print("%d cols" %(col))

market_na = pd.DataFrame(market.isnull().sum(), columns = ["NA"])
print("Features with missing values greater than 85% of total:")
na_feat_list = (market_na[market_na['NA'] > ((row * 85)/100)])
na_feat_list = na_feat_list.index.values
market = market.drop(na_feat_list,axis = 1)



462298 rows
76 cols
Features with missing values greater than 85% of total:


## General informations

#### General finding by feature

* Portfolio1- de ramo: COMERCIO VAREJISTA  total = 496, BENS DE CONSUMO   total = 59
* Portfolio1- de_natureza_juridica: Empresario Individual, Portfolio2: Sociedade Limitada e Portfolio3: Município
* Portfolio3- setor: Serviço
* Portfolio1- idade_emp_cat: 1 a 5 e 5 a 10 anos, predominante. 
* Portfolio1- fl_mei significativo. Provavelmente devido ao maior numero de empresarios individuais.
* Portfolio2- de_nivel_atividade predominante, Portfolio1 possui empresas de nível de atividade Muito Baixa.
* Portfolio1- vl_faturamento_estimado_aux menor media de faturamento, Portfolio3 maior média de faturamento


In [305]:
 market[market['portfolio'] == 2]['tx_rotatividade'].mean()

15.720659504914359


## Dataset separado em :
* 1. Valores não-numéricos
* 2. Valores Numéricos
* 3. Booleanos

In [70]:
market_object= market.select_dtypes(include = ['object'])
market_object.head()

,id,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,dt_situacao,fl_rm,nm_divisao,...,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,10 a 15,2005-03-25,SIM,CONSTRUCAO DE EDIFICIOS,...,RN,VERDE,ACIMA DE 1 ANO,ALTA,False,LESTE POTIGUAR,NATAL,True,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00"
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1 a 5,2017-05-12,SIM,ALIMENTACAO,...,PI,CINZA,ATE 1 ANO,BAIXA,False,CENTRO NORTE PIAUIENSE,TERESINA,True,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00"
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,5 a 10,2011-09-26,SIM,TRANSPORTE TERRESTRE,...,AM,AMARELO,ACIMA DE 1 ANO,MEDIA,False,CENTRO AMAZONENSE,MANAUS,True,"ATE R$ 81.000,00","ATE R$ 81.000,00"
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,5 a 10,2012-04-25,SIM,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,...,AM,AMARELO,ACIMA DE 1 ANO,MEDIA,False,CENTRO AMAZONENSE,MANAUS,True,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00"
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,1 a 5,2015-08-17,SIM,SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,...,RN,VERDE,ACIMA DE 1 ANO,ALTA,False,LESTE POTIGUAR,NATAL,True,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00"


In [71]:
market_num= market.select_dtypes(include = ['float64', 'int64'])
market_num.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,label
0,0,14.457534,0.0,0.0,23.0,NaN,2.0,2.0,0.0,44.0,...,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0,0
1,1,1.463014,0.0,0.0,9.0,468.93,1.0,1.0,0.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,7.093151,0.0,0.0,26.0,713.50,1.0,1.0,0.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,6.512329,0.0,0.0,26.0,475.73,1.0,1.0,0.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,3.200000,0.0,0.0,27.0,2874.04,1.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [73]:
market_bool= market.select_dtypes(include = ['bool'])
market_bool.head()

,fl_matriz,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone
0,True,False,False,False,False,False,False,True,True
1,True,False,False,False,True,False,False,True,True
2,True,False,False,False,True,False,False,False,True
3,True,False,False,False,False,False,False,True,True
4,True,False,False,False,False,False,False,True,True


In [72]:
portfolio_collection = market_num[(market_num['portfolio'] == 1) |( market_num['portfolio'] == 2)|( market_num['portfolio'] == 3)]
portfolio_collection.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,label
369,369,0.646575,0.0,0.0,NaN,379.96,1.0,1.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
604,604,6.586301,0.0,0.0,22.0,1865.61,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,2.0,NaN,-100.000000,0.0,4,1
1646,1646,11.243836,0.0,4.0,23.0,2791.15,NaN,NaN,NaN,NaN,...,357.066667,25.533333,1361.0,1361.0,1383.0,0.0,-1.590745,0.0,0,2
1703,1703,8.010959,0.0,0.0,24.0,798.12,1.0,1.0,0.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2521,2521,20.863014,0.0,0.0,26.0,608.47,1.0,1.0,0.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


# Pipeline

In [96]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(portfolio_collection.drop('portfolio', axis = 1))
portfolio_transformed =  pd.DataFrame(pipeline_transformation, columns=portfolio_collection.drop('portfolio', axis = 1).columns)
portfolio_transformed['portfolio'] = portfolio_collection['portfolio'].tolist()
portfolio_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,label
0,0.000000,0.010795,0.0,0.000000,0.340426,0.032268,0.000000,0.000000,0.0,0.476744,...,0.048495,0.009659,0.008049,0.007747,0.008585,0.007812,0.005257,0.016528,0.000000,1
1,0.000509,0.118645,0.0,0.000000,0.319149,0.210046,0.007246,0.007246,0.0,0.493023,...,0.048495,0.009659,0.000000,0.000000,0.000109,0.007812,0.000000,0.000000,0.010695,1
2,0.002765,0.203214,0.0,0.010782,0.340426,0.320800,0.007246,0.007246,0.0,0.493023,...,0.068967,0.057957,0.075813,0.075851,0.075384,0.007812,0.005498,0.000000,0.000000,2
3,0.002889,0.144513,0.0,0.000000,0.361702,0.082307,0.000000,0.000000,0.0,0.581395,...,0.048495,0.009659,0.008049,0.007747,0.008585,0.007812,0.005257,0.016528,0.000000,1
4,0.004660,0.377873,0.0,0.000000,0.404255,0.059613,0.000000,0.000000,0.0,0.418605,...,0.048495,0.009659,0.008049,0.007747,0.008585,0.007812,0.005257,0.016528,0.000000,1


### QUANTILES
* Lower and uppper limits to get number of outliers of numeric feat

In [97]:
def get_outliers_limit(dataframe):
    column_name = dataframe.columns 
    IQR=[]
    Q1= []
    Q3= []

    lw_limit = []
    up_limit = []
    for col in column_name:
        q_1 = np.percentile(dataframe[col], 25)
        q_3 = np.percentile(dataframe[col], 75)

        Q1.append(q_1)
        Q3.append(q_3)  
        IQR.append(q_3-q_1)

    for i in range(0,len(IQR)):
        lw_limit.append((Q1[i] - 1.5*IQR[i]))
        up_limit.append((Q3[i] + 1.5*IQR[i]))
    
    return(lw_limit,up_limit)

lower_limit,upper_limit = get_outliers_limit(portfolio_transformed)
data = {'lower_limit':lower_limit, 'upper_limit':upper_limit}
outliers = pd.DataFrame([lower_limit,upper_limit],columns= portfolio_transformed.columns)
outliers.dropna(axis=1)

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,label
0,-0.524693,-0.422632,-0.004525,-0.020216,0.308511,-0.044842,-0.010870,-0.010870,0.0,0.360465,...,0.003424,-0.066869,-0.003314,-0.003846,-0.005096,0.007812,0.004132,-0.077151,-0.004011,-0.5
1,1.503762,1.069259,0.007541,0.033693,0.393617,0.294435,0.018116,0.018116,0.0,0.639535,...,0.088484,0.121327,0.020416,0.020677,0.024229,0.007812,0.006308,0.128584,0.006684,3.5


# One hot encoding

In [25]:
#enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])

[0. 0. 1. 0. 0. 0.]


# PCA

In [308]:
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(portfolio_transformed)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])

In [309]:
principalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
0,-0.838736,0.079669,-0.493785,-0.034904,-0.122872
1,-0.821333,0.053561,-0.491185,-0.038067,-0.021400
2,0.175134,-0.337739,-0.507323,0.173282,-0.152170
3,-0.801404,0.081470,-0.485452,0.010144,0.089183
4,-0.773788,0.141224,-0.487163,0.029205,0.070985
